In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import path
import matplotlib.colors as mcolors
import geopandas as gpd
import scipy.io as sio
from scipy.interpolate import griddata

import colorcet as cc
from mpl_toolkits.axes_grid1 import make_axes_locatable

%config InlineBackend.figure_format = "retina"

In [2]:
def inpolygon(xq, yq, xv, yv):
    shape = xq.shape
    xq = xq.reshape(-1)
    yq = yq.reshape(-1)
    xv = xv.reshape(-1)
    yv = yv.reshape(-1)
    q = [(xq[i], yq[i]) for i in range(xq.shape[0])]
    p = path.Path([(xv[i], yv[i]) for i in range(xv.shape[0])])
    return p.contains_points(q).reshape(shape)

# Hyuganada Sea (2024)

In [ ]:
# Read 2024 Kyushu earthquake finite fault model from:
# https://earthquake.usgs.gov/realtime/product/finite-fault/us6000jllz_1/us/1676400595497/FFM.geojson
coseismic_file_name = "hyuganada_sea_FFM.geojson"
coseismic_data = gpd.read_file(coseismic_file_name)

# Extract mean longitude, latitude, and depth from each rectangle
mean_longitude = np.zeros(len(coseismic_data))
mean_latitude = np.zeros(len(coseismic_data))
mean_depth = np.zeros(len(coseismic_data))
for i in range(len(coseismic_data)):
    polygon = coseismic_data["geometry"][i]
    coords = np.array(list(polygon.exterior.coords))
    mean_longitude[i] = np.mean(coords[0:4].T[0])
    mean_latitude[i] = np.mean(coords[0:4].T[1])
    mean_depth[i] = np.mean(coords[0:4].T[2])

# USGS Kyushu coseismic
lon_vec = np.linspace(130, 133, 1000)
lat_vec = np.linspace(30, 33, 1000)
lon_grid, lat_grid = np.meshgrid(lon_vec, lat_vec)
points = np.array([mean_longitude, mean_latitude]).T
grid_coseismic = griddata(
    points, coseismic_data["slip"].values, (lon_grid, lat_grid), method="cubic"
)

# Loveless and Meade (2010) interseismic
df = pd.read_csv("loveless_meade_coupling.csv")

points = np.array([df.lon.values, df.lat.values]).T
grid_interseismic = griddata(
    points, df.ds_sd.values, (lon_grid, lat_grid), method="cubic"
)


# plt.figure(figsize=(5, 5))

fig, ax = plt.subplots(figsize=(5, 5))

WORLD_BOUNDARIES = sio.loadmat("WorldHiVectors.mat")
plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color="lightgray",
    linewidth=0.0,
    edgecolor="none",
    zorder=1,
)

plt.xlim([130, 133])
plt.ylim([30, 33])
plt.xticks([130, 133])
plt.yticks([30, 33])
# plt.xticks([130, 131, 132, 133])
# plt.yticks([30, 31, 32, 33])
plt.gca().set_aspect("equal")
plt.xlabel("longitude")
plt.ylabel("latitude")

grid_interseismic_plot = np.copy(grid_interseismic)
grid_interseismic_plot[grid_interseismic_plot > 50] = 50
grid_interseismic_plot[grid_interseismic_plot < -50] = -50

f = grid_interseismic_plot.flatten()
small_idx_1 = np.where(f < 5)[0]
small_idx_2 = np.where(f > -5)[0]
small_idx = np.intersect1d(small_idx_1, small_idx_2)
f[small_idx] = np.nan
grid_interseismic_plot = np.reshape(f, (1000, 1000))

levels = np.linspace(-50, 50, 9)
contour1 = plt.contourf(
    lon_grid,
    lat_grid,
    grid_interseismic_plot,
    levels=levels,
    cmap=cc.cm.bwy,
    vmin=-50.0,
    vmax=50.0,
    zorder=2,
    alpha=1.0,
)

levels = np.linspace(0.25, 2.5, 5)
grid_coseismic_plot = np.copy(grid_coseismic)
grid_coseismic_plot[grid_coseismic_plot > 2.5] = 2.5
grid_coseismic_plot[grid_coseismic_plot < 0.25] = np.nan

contour2 = plt.contourf(
    lon_grid,
    lat_grid,
    grid_coseismic_plot,
    levels=levels,
    cmap=cc.cm.CET_L4_r,
    vmin=0.25,
    vmax=2.5,
    zorder=3,
)


# Create a divider for the axis to append colorbars
divider = make_axes_locatable(ax)


# Add the first colorbar to the right of the plot
cax1 = divider.append_axes("right", size="5%", pad=0.15)
# # cbar1 = plt.colorbar(contour1, cax=cax1)
cbar1 = plt.colorbar(
    contour1,
    cax=cax1,
    extend="both",
    label="slip deficit rate (mm/yr)",
    fraction=0.046,
    pad=0.04,
)
cbar1.set_ticks([-50, 0, 50])

# # Add a second colorbar to the right of the first colorbar
cax2 = divider.append_axes(
    "right", size="5%", pad=0.9
)  # Increased padding to space colorbars
cbar2 = plt.colorbar(contour2, cax=cax2)
cbar2.set_label("coseismic slip (m)")
cbar2.set_ticks([0.25, 2.5])

width, height = fig.get_size_inches()
print(f"Figure height: {height} inches")

plt.savefig("kyushu_colocation.pdf", bbox_inches="tight")
plt.savefig("kyushu_colocation.png", dpi=500, bbox_inches="tight")
plt.show()

# Kyushu: Find interseismic slip deficit rates inside of minimum slip contour
lon_grid_kyushu = copy.deepcopy(lon_grid)
lat_grid_kyushu = copy.deepcopy(lat_grid)
grid_interseismic_kyushu = copy.deepcopy(grid_interseismic)
grid_coseismic_kyushu = copy.deepcopy(grid_coseismic)
idx_in_kyushu = np.where(grid_coseismic_kyushu.flatten() > 0.25)[0]

# Tohoku (2010)

In [ ]:
# Read 2024 Kyushu earthquake finite fault model from:
# https://earthquake.usgs.gov/realtime/product/finite-fault/us6000jllz_1/us/1676400595497/FFM.geojson
coseismic_file_name = "tohoku_FFM.geojson"
coseismic_data = gpd.read_file(coseismic_file_name)

# Extract mean longitude, latitude, and depth from each rectangle
mean_longitude = np.zeros(len(coseismic_data))
mean_latitude = np.zeros(len(coseismic_data))
mean_depth = np.zeros(len(coseismic_data))
for i in range(len(coseismic_data)):
    polygon = coseismic_data["geometry"][i]
    coords = np.array(list(polygon.exterior.coords))
    mean_longitude[i] = np.mean(coords[0:4].T[0])
    mean_latitude[i] = np.mean(coords[0:4].T[1])
    mean_depth[i] = np.mean(coords[0:4].T[2])

# USGS Kyushu coseismic
lon_vec = np.linspace(140, 146, 1000)
lat_vec = np.linspace(35, 41, 1000)
lon_grid, lat_grid = np.meshgrid(lon_vec, lat_vec)
points = np.array([mean_longitude, mean_latitude]).T
grid_coseismic = griddata(
    points, coseismic_data["slip"].values, (lon_grid, lat_grid), method="cubic"
)

# Loveless and Meade (2010) interseismic
df = pd.read_csv("loveless_meade_coupling.csv")

points = np.array([df.lon.values, df.lat.values]).T
grid_interseismic = griddata(
    points, df.ds_sd.values, (lon_grid, lat_grid), method="cubic"
)


fig, ax = plt.subplots(figsize=(5, 5))
WORLD_BOUNDARIES = sio.loadmat("WorldHiVectors.mat")
plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color="lightgray",
    linewidth=0.0,
    edgecolor="none",
    zorder=1,
)

plt.xlim([140, 146])
plt.ylim([35, 41])
plt.xticks([140, 146])
plt.yticks([35, 41])

# plt.xticks([130, 131, 132, 133])
# plt.yticks([30, 31, 32, 33])
plt.gca().set_aspect("equal")
plt.xlabel("longitude")
plt.ylabel("latitude")

grid_interseismic_plot = np.copy(grid_interseismic)
grid_interseismic_plot[grid_interseismic_plot > 90] = 90
grid_interseismic_plot[grid_interseismic_plot < -90] = -90

f = grid_interseismic_plot.flatten()
small_idx_1 = np.where(f < 15)[0]
small_idx_2 = np.where(f > -15)[0]
small_idx = np.intersect1d(small_idx_1, small_idx_2)
f[small_idx] = np.nan
grid_interseismic_plot = np.reshape(f, (1000, 1000))

levels = np.linspace(-90, 90, 9)
contour1 = plt.contourf(
    lon_grid,
    lat_grid,
    grid_interseismic_plot,
    levels=levels,
    cmap=cc.cm.bwy,
    vmin=-90.0,
    vmax=90.0,
    zorder=2,
    alpha=1.0,
)

# levels = np.linspace(0.1, 2.5, 5)
# levels = 5
levels = np.linspace(5.5, 55, 5)

grid_coseismic_plot = np.copy(grid_coseismic)
grid_coseismic_plot[grid_coseismic_plot > 55] = 55
grid_coseismic_plot[grid_coseismic_plot < 5.5] = np.nan

contour2 = plt.contourf(
    lon_grid,
    lat_grid,
    grid_coseismic_plot,
    levels=levels,
    cmap=cc.cm.CET_L4_r,
    vmin=5.5,
    vmax=55,
    zorder=3,
)

# Create a divider for the axis to append colorbars
divider = make_axes_locatable(ax)

# Add the first colorbar to the right of the plot
cax1 = divider.append_axes("right", size="5%", pad=0.15)
cbar1 = plt.colorbar(
    contour1,
    cax=cax1,
    extend="both",
    label="slip deficit rate (mm/yr)",
    fraction=0.046,
    pad=0.04,
)
cbar1.set_ticks([-90, 0, 90])


# Add a second colorbar to the right of the first colorbar
cax2 = divider.append_axes(
    "right", size="5%", pad=0.9
)  # Increased padding to space colorbars
cbar2 = plt.colorbar(contour2, cax=cax2)
cbar2.set_label("coseismic slip (m)")
cbar2.set_ticks([5.5, 55])

plt.savefig("tohoku_colocation.pdf", bbox_inches="tight")
plt.savefig("tohoku_colocation.png", dpi=500, bbox_inches="tight")
plt.show()

# Tohoku: Find interseismic slip deficit rates inside of minimum slip contour
lon_grid_tohoku = copy.deepcopy(lon_grid)
lat_grid_tohoku = copy.deepcopy(lat_grid)
grid_interseismic_tohoku = copy.deepcopy(grid_interseismic)
grid_coseismic_tohoku = copy.deepcopy(grid_coseismic)
idx_in_tohoku = np.where(grid_coseismic_tohoku.flatten() > 5)[0]

In [ ]:
tohoku_x = grid_interseismic_tohoku.flatten()[idx_in_tohoku]
# tohoku_x -= np.min(tohoku_x)
tohoku_x /= np.nanmax(grid_interseismic_tohoku.flatten())
tohoku_y = grid_coseismic_tohoku.flatten()[idx_in_tohoku]
tohoku_y -= np.nanmin(tohoku_y)
tohoku_y /= np.nanmax(tohoku_y)

kyushu_x = grid_interseismic_kyushu.flatten()[idx_in_kyushu]
# kyushu_x -= np.nanmin(kyushu_x)
kyushu_x = kyushu_x / np.nanmax(grid_interseismic_kyushu.flatten())
kyushu_y = grid_coseismic_kyushu.flatten()[idx_in_kyushu]
kyushu_y -= np.nanmin(kyushu_y)
kyushu_y /= np.nanmax(kyushu_y)

fig = plt.figure(figsize=(7, 3.5))
# plt.fill([-1, 0, 0, -1], [0, 0, 1, 1], color="lightgrey", zorder=1)

counts_kyushu, xedges_kyushu, yedges_kyushu, h_kyushu = plt.hist2d(
    kyushu_x,
    kyushu_y,
    bins=[np.linspace(-1, 1, 21), np.linspace(0, 1, 11)],
    cmap="Blues",
    norm=mcolors.LogNorm(),
    zorder=2,
    alpha=0.75,
    label="$\mathrm{M}_\mathrm{W} = 7.1$ Kyushu (2024)",
)

counts_tohoku, xedges_tohoku, yedges_tohoku, h_tohoku = plt.hist2d(
    tohoku_x,
    tohoku_y,
    bins=[np.linspace(-1, 1, 21), np.linspace(0, 1, 11)],
    # cmap="plasma_r",
    cmap="Reds",
    norm=mcolors.LogNorm(),
    zorder=2,
    alpha=0.75,
    label="$\mathrm{M}_\mathrm{W}=9.0$, Tohoku (2010)",
)

plt.plot([0, 0], [0, 1], "--k", zorder=3)
plt.text(-0.97, 0.93, "$\mathrm{M}_\mathrm{W}=9.0$ Tohoku (2010)", color="red")
plt.text(-0.97, 0.85, "$\mathrm{M}_\mathrm{W}=7.1$ Kyushu (2024)", color="blue")
plt.xlim([-1, 1])
plt.ylim([0, 1])
plt.xticks([-1.0, -0.5, 0.0, 0.5, 1.0])
plt.yticks([0.0, 0.5, 1.0])
plt.gca().set_aspect("equal")
plt.xlabel("normalized interseismic slip deficit rate")
plt.ylabel("normalized coseismic slip")

x_delta = 0.30
fig.text(0.32, -0.04, "moment release", ha="center", va="center", fontsize=10)
fig.text(0.7, -0.04, "moment accumulation", ha="center", va="center", fontsize=10)

plt.savefig("compare_colocation.pdf", bbox_inches="tight")
plt.savefig("compare_colocation.png", dpi=500, bbox_inches="tight")

plt.show()

In [ ]:
fig = plt.figure(figsize=(7, 7))
# plt.fill([-1, 0, 0, -1], [0, 0, 1, 1], color="lightgrey", zorder=1)

plt.subplot(2, 1, 1)
counts_tohoku, xedges_tohoku, yedges_tohoku, h_tohoku = plt.hist2d(
    tohoku_x,
    tohoku_y,
    bins=[np.linspace(-1, 1, 101), np.linspace(0, 1, 51)],
    # cmap="plasma_r",
    cmap="Reds",
    norm=mcolors.LogNorm(),
    zorder=2,
    # alpha=0.75,
    label="$\mathrm{M}_\mathrm{W}=9.0$, Tohoku (2010)",
)

plt.plot([0, 0], [0, 1], "--k", zorder=3)
plt.text(-0.97, 0.85, "Tohoku-oki\n2010, $\mathrm{M}_\mathrm{W}=9.1$", color="red")
# plt.text(-0.97, 0.85, "$\mathrm{M}_\mathrm{W}=7.1$ Kyushu (2024)", color="blue")
plt.xlim([-1, 1])
plt.ylim([0, 1])
# plt.xticks([-1.0, -0.5, 0.0, 0.5, 1.0])
plt.xticks([])
plt.yticks([0.0, 0.5, 1.0])
plt.gca().set_aspect("equal")
# plt.xlabel("normalized interseismic slip deficit rate")
plt.ylabel("normalized coseismic slip")


plt.subplot(2, 1, 2)
counts_kyushu, xedges_kyushu, yedges_kyushu, h_kyushu = plt.hist2d(
    kyushu_x,
    kyushu_y,
    bins=[np.linspace(-1, 1, 101), np.linspace(0, 1, 51)],
    cmap="Blues",
    norm=mcolors.LogNorm(),
    zorder=2,
    # alpha=0.75,
    label="$\mathrm{M}_\mathrm{W} = 7.1$ Hyuganada Sea \n (2024)",
)

plt.plot([0, 0], [0, 1], "--k", zorder=3)
# plt.text(-0.97, 0.93, "$\mathrm{M}_\mathrm{W}=9.0$ Tohoku (2010)", color="red")
plt.text(-0.97, 0.85, "Hyuganada Sea\n2024, $\mathrm{M}_\mathrm{W}=7.1$", color="blue")
plt.xlim([-1, 1])
plt.ylim([0, 1])
plt.xticks([-1.0, -0.5, 0.0, 0.5, 1.0])
plt.yticks([0.0, 0.5, 1.0])
plt.gca().set_aspect("equal")
plt.xlabel("normalized interseismic slip deficit rate")
plt.ylabel("normalized coseismic slip")

x_delta = 0.30
fig.text(0.34, 0.04, "moment release", ha="center", va="center", fontsize=10)
fig.text(0.68, 0.04, "moment accumulation", ha="center", va="center", fontsize=10)
plt.savefig("compare_colocation_two_panel.pdf", bbox_inches="tight")
plt.savefig("compare_colocation_two_panel.png", dpi=500, bbox_inches="tight")

plt.show()

In [ ]:
np.nanmax(grid_coseismic)

percentage_kyushu = np.mean(kyushu_x < 0) * 100
print(f"Kyushu (2024): percentage of elements less than zero: {percentage_kyushu:.2f}%")

percentage_tohoku = np.mean(tohoku_x > 0) * 100
print(
    f"Tohoku (2010): percentage of elements greater than zero: {percentage_tohoku:.2f}%"
)

In [ ]:
print(np.corrcoef(tohoku_x, tohoku_y))
print(np.corrcoef(kyushu_x, kyushu_y))

In [ ]:
fs = 3.4

fig, ax = plt.subplots(figsize=(fs, fs))
WORLD_BOUNDARIES = sio.loadmat("WorldHiVectors.mat")
plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color="lightgray",
    linewidth=0.0,
    edgecolor="none",
    zorder=1,
)
plt.plot([130, 133, 133, 130, 130], [30, 30, 33, 33, 30], "--k")
plt.xlim([129, 146])
plt.ylim([30, 47])
plt.xticks([129, 146])
plt.yticks([30, 47])
plt.gca().set_aspect("equal")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.savefig("japan_kyushu_colocation.pdf", bbox_inches="tight")
plt.savefig("japan_kyushu_colocation.png", dpi=500, bbox_inches="tight")
plt.show()


fig, ax = plt.subplots(figsize=(fs, fs))
WORLD_BOUNDARIES = sio.loadmat("WorldHiVectors.mat")
plt.fill(
    WORLD_BOUNDARIES["lon"],
    WORLD_BOUNDARIES["lat"],
    color="lightgray",
    linewidth=0.0,
    edgecolor="none",
    zorder=1,
)

# plt.xlim([140, 146])
# plt.ylim([35, 41])
plt.plot([140, 146, 146, 140, 140], [35, 35, 41, 41, 35], "--k")
plt.xlim([129, 146])
plt.ylim([30, 47])
plt.xticks([129, 146])
plt.yticks([30, 47])
plt.gca().set_aspect("equal")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.savefig("japan_tohoku_colocation.pdf", bbox_inches="tight")
plt.savefig("japan_tohoku_colocation.png", dpi=500, bbox_inches="tight")
plt.show()

width, height = fig.get_size_inches()
print(f"Figure height: {height} inches")

In [ ]:
from scipy.stats import pearsonr
import numpy as np

# Example data arrays
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 6, 8, 10])

x = tohoku_x
y = tohoku_y

# Calculate the Pearson correlation coefficient
correlation_coefficient, _ = pearsonr(x, y)

print(f"Correlation Coefficient: {correlation_coefficient:.4f}")


# Bootstrap method to estimate uncertainty
def bootstrap_corr(x, y, n_bootstraps=1000):
    rng = np.random.default_rng()
    bootstrap_corrs = []
    for _ in range(n_bootstraps):
        indices = rng.integers(0, len(x), len(x))
        x_sample = x[indices]
        y_sample = y[indices]
        bootstrap_corrs.append(pearsonr(x_sample, y_sample)[0])
    return np.std(bootstrap_corrs)


# Estimate uncertainty
uncertainty = bootstrap_corr(x, y)

print(f"Estimated Uncertainty: {uncertainty:.4f}")